# Python Data Analysis of Titanic Survival with Pandas

In this project, we will analyze the open dataset with data on the passengers aboard the Titanic. The main purpose is to analyze, clean and transform the data to answer the following question: 

> What categories of passengers were most likely to survive the Titanic disaster?

The data file `train.csv`, renamed to `titanic_data.csv` here, can be downloaded from Kaggle [here](https://www.kaggle.com/c/titanic/data). The definition of all variables can be found on the same Kaggle page, in the Data Dictionary section.

**Section 01.**
* We will conduct a detailed analysis to identify what categories of passengers were most likely to survive the Titanic disaster.
* We will also learn about the logic of the analysis.

**Section 02.**
* For all the attributes used for the analysis in the previous section, we will see why and how we used them.

**Section 03.**
* There were missing values that were not taken into consideration previously. They could have impacted our analysis. We will see how our analysis has changed after dropping the rows with missing values in this section.


---

## Section 01

In [1]:
import pandas as pd

In [2]:
## Read the data file into a DataFrame
df = pd.read_csv("titanic_data.csv")

In [3]:
# print(df)
print(df.head())
print(df.shape)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
(8

In [4]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [5]:
## Check the data types of all columns
print(df.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [6]:
## Show descriptive statistics
print(df.describe())
print(df.describe(include='all'))

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
        PassengerId    Survived      Pclass                     Name   Sex  \
cou

### Analysis 

#### 1) Ticket Class

According to [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data), `Pclass` refers to ticket class with 1st being the upper class and 3rd the lower class. 

In [7]:
print(df.groupby("Pclass")["Survived"].count())

Pclass
1    216
2    184
3    491
Name: Survived, dtype: int64


The lower class, 3rd class ticket holders, was the most likely to survive the Titanic disaster. It is almost more than double of the next group most likely to survive - 1st class. 

#### 2) Sex

`Sex` is binary - only `male` and `female`. 

In [8]:
print(df.groupby("Sex")["Survived"].count())

Sex
female    314
male      577
Name: Survived, dtype: int64


Male passengers are more likely to survive the disaster, almost twice as much as female passengers. 

#### 3) Age

Age will be grouped according to the different age groups set by [Statistics Canada](https://www.statcan.gc.ca/en/concepts/definitions/previous/age1a) but with the adult group further divided into young and middle-aged adult groups. See the groupings below.

- Children (00-14 years)
- Youth (15-24 years)
- Young Adults (25-40 years)
- Middle-aged Adults (41-64 years)
- Seniors (65 years and over)

In [9]:
print("Oldest passenger:", df["Age"].max(), "years old")
print("Youngest passenger:", df["Age"].min(), "years old")

Oldest passenger: 80.0 years old
Youngest passenger: 0.42 years old


In [10]:
## Sort age values to groups of age ranges
bins = [0, 15, 25, 41, 65, 100]
group_names = ["Children", "Youth", "Young Adults", "Middle-aged Adults", "Seniors"]
df['Age Groups'] = pd.cut(df["Age"], bins, labels=group_names)
# print(df[["Age Groups", "Survived"]])
df.groupby("Age Groups")["Survived"].count()

Age Groups
Children               83
Youth                 218
Young Adults          271
Middle-aged Adults    134
Seniors                 8
Name: Survived, dtype: int64

Young adults are most likely to survive, followed by youth, but seniors are the least likely to survive. 

#### 4) Fare

Ticket fares will be grouped into equal intervals.

In [11]:
fare_lowest = df["Fare"].max()
print(f"Most expensive fare: ${fare_lowest}")
fare_highest = df["Fare"].min()
print(f"Least expensive fare: ${fare_highest}")

Most expensive fare: $512.3292
Least expensive fare: $0.0


In [12]:
## Discretize variable into equal-sized buckets based on rank or sample quantiles
bin_labels_3 = ["Low", "Medium", "High"]
q3 = [0, 1/3, 2/3, 1]
df["Fare Range"] = pd.qcut(df["Fare"], q=q3, labels=bin_labels_3)
# bin_labels_4 = ["Quartile 1", "Quartile 2", "Quartile 3", "Quartile 4"]
# q4 = [0, 0.25, 0.5, 0.75, 1]
# df["Fare Range"] = pd.qcut(df["Fare"], q=q4, labels=bin_labels_4)
# bin_labels_6 = ["Low Low", "Low High", "Medium Low", "Medium High", "High Low", "High High"]
# q6 = [0, 1/6, 2/6, .5, 4/6, 5/6, 1]
# df["Fare Range"] = pd.qcut(df["Fare"], q=q6, labels=bin_labels_6)
print(df["Fare Range"].value_counts())

Low       308
High      295
Medium    288
Name: Fare Range, dtype: int64


Those passengers who bought the lowest-tier tickets were most likely to survive. But the difference between ticket tiers does not seem to be striking in terms of survival rate.

#### 5) Embarkation

`Embarked` refers to ports of Embarkation. 

- C = Cherbourg
- Q = Queenstown
- S = Southampton

In [13]:
print(df.groupby("Embarked")["Survived"].count())

Embarked
C    168
Q     77
S    644
Name: Survived, dtype: int64


Passengers who embarked at Southampton were significantly more likely to survive the disaster but those at Queenstown had the lowest chance of survival. 

### Logic of Analysis

First of all, we will go through all the attributes and eliminate those not relevant to our analysis. Only those that seem to be relevant are selected: 

- Ticket Class: `Pclass`
- Sex: `Sex`
- Age: `Age` and `Age Groups`
- Fare: `Fare` and `Fare Range`
- Embarkation: `Embarked`

Analysis is made simpler by analyzing those category one by one. For categorial data (`Pclass`, `Sex` and `Embarked`), we can perform simple analysis by grouping data in thsoe columns and count the `Survived` indicator of 1. However, for those with continuous numeric data (`Age` and `Fare`), we should define ranges and intervals for grouping first. 

## Section 02

In [14]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age Groups',
       'Fare Range'],
      dtype='object')


In [15]:
## Check the data types of all columns
print(df.dtypes)

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked         object
Age Groups     category
Fare Range     category
dtype: object


Attributes used for analysis: 

```python
["Pclass", "Sex", "Age", "Fare", "Embarked"]
```

### Why & How

`Age Groups` and `Fare Range` are created from the original attributes `['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']`

Why:

- `PassengerId` and `Name` are randomly assigned to passengers and therefore not useful in telling the survival rate.
- `Survived` is a crucial attribute in knowing whether the passengers survived in the disaster. 
- `Pclass` might be a factor as it reflects the social status of the passengers.
- `Sex` and `Age` are indicators of physical strength and so may affect the survival rate in a life-or-death situation.
- `Ticket` and `Cabin` seem to be randomly coded and assigned so it does not give much information about survival rate.
- `Fare`, however, is similar to `Pclass` in the sense that it indicates the economic status of the passengers.
- `Embarked` can somehow tell the social status of the passengers since people of different social classes live in different areas and they tend to embark at the nearest port to their residential area.

How: 

- `Survived` is binary coded - only 1 and 0 - so we can simply count rows where it is coded as `1` when analyzing different categories.
- `Pclass`, `Sex` and `Embarked` are categorial data and so we can just use `groupby()` to classify the data.
- `Age` and `Fare` are continuous numeric data and just use `groupby()` to group the values would be useless. Therefore, we should create age groups (`Age Groups`) and divide fare into tiers (`Fare Range`) before counting the data ranges.

## Section 03

### Data Cleaning

We can drop rows with all or any NA values. 

In [16]:
## Find rows with all NA values
## isna() - returns a boolean DataFrame that indicates whether each element is NA
## any(axis=1) - reduces columns of the boolean DataFrame to only a boolean Series
print(df.isna())
print(df.isna().any(axis=1))
df_nan = df[df.isna().any(axis=1)]
print(df_nan.head())
print(f"Number of rows: {df_nan.shape[0]}")

## Drop rows with all NA values
df.dropna(how='all', inplace=True)
# print(df.head())
print(f"Number of rows: {df.shape[0]}")

     PassengerId  Survived  Pclass   Name    Sex    Age  SibSp  Parch  Ticket  \
0          False     False   False  False  False  False  False  False   False   
1          False     False   False  False  False  False  False  False   False   
2          False     False   False  False  False  False  False  False   False   
3          False     False   False  False  False  False  False  False   False   
4          False     False   False  False  False  False  False  False   False   
..           ...       ...     ...    ...    ...    ...    ...    ...     ...   
886        False     False   False  False  False  False  False  False   False   
887        False     False   False  False  False  False  False  False   False   
888        False     False   False  False  False   True  False  False   False   
889        False     False   False  False  False  False  False  False   False   
890        False     False   False  False  False  False  False  False   False   

      Fare  Cabin  Embarked

After attempting to drop rows with all NA values, the number of rows of the DataFrame is still the same as before. Therefore, there is no row with all NA values. 

If we drop rows with any value in the original DataFrame, there will only be 183 rows left for analysis, which is not representative. See below.

In [17]:
## Drop rows with any NA values
df = df.dropna(how='any', inplace=False)
# print(df.head())
print(f"Number of rows: {df.shape[0]}")

Number of rows: 183


As some of the attributes do not matter in our survival rate analysis, we will create a new DataFrame selecting only those attributes that are potential factors in our analysis. See below.

In [18]:
df_factors = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Age Groups', 'Fare Range']]
print(df_factors)

     Survived  Pclass     Sex   Age     Fare Embarked          Age Groups  \
1           1       1  female  38.0  71.2833        C        Young Adults   
3           1       1  female  35.0  53.1000        S        Young Adults   
6           0       1    male  54.0  51.8625        S  Middle-aged Adults   
10          1       3  female   4.0  16.7000        S            Children   
11          1       1  female  58.0  26.5500        S  Middle-aged Adults   
..        ...     ...     ...   ...      ...      ...                 ...   
871         1       1  female  47.0  52.5542        S  Middle-aged Adults   
872         0       1    male  33.0   5.0000        S        Young Adults   
879         1       1  female  56.0  83.1583        C  Middle-aged Adults   
887         1       1  female  19.0  30.0000        S               Youth   
889         1       1    male  26.0  30.0000        C        Young Adults   

    Fare Range  
1         High  
3         High  
6         High  
10     

Now with the new DataFrame, we have much more data to do the analysis. We only need to drop those rows with NA values in one of those columns that will potentially affect our analysis.

In [19]:
## Drop rows with all NA values
df_factors = df_factors.dropna(how='any', inplace=False)
# print(df.head())
print(f"Number of rows: {df_factors.shape[0]}")

Number of rows: 183


Now we will perform the same analysis with the new DataFrame as above. 

### New Analysis 

#### 1) Ticket Class

In [20]:
print(df_factors.groupby("Pclass")["Survived"].count())

Pclass
1    158
2     15
3     10
Name: Survived, dtype: int64


##### Previous Analysis

The lower class, 3rd class ticket holders, was the most likely to survive the Titanic disaster. It is almost more than double of the next group most likely to survive - 1st class. 

##### New Analysis

Now the upper class is the most likely to survive.

#### 2) Sex

In [21]:
print(df_factors.groupby("Sex")["Survived"].count())

Sex
female    88
male      95
Name: Survived, dtype: int64


##### Previous Analysis

Male passengers are more likely to survive the disaster, almost twice as much as female passengers. 

##### New Analysis

Same as before removing those NA values.

#### 3) Age

In [22]:
print("Oldest passenger:", df_factors["Age"].max(), "years old")
print("Youngest passenger:", df_factors["Age"].min(), "years old")

Oldest passenger: 80.0 years old
Youngest passenger: 0.92 years old


In [23]:
## Sort age values to groups of age ranges
bins = [0, 15, 25, 41, 65, 100]
group_names = ["Children", "Youth", "Young Adults", "Middle-aged Adults", "Seniors"]
df_factors['Age Groups'] = pd.cut(df_factors["Age"], bins, labels=group_names)
# print(df[["Age Groups", "Survived"]])
df_factors.groupby("Age Groups")["Survived"].count()

Age Groups
Children              13
Youth                 39
Young Adults          67
Middle-aged Adults    61
Seniors                3
Name: Survived, dtype: int64

##### Previous Analysis

Young adults are most likely to survive, followed by youth, but seniors are the least likely to survive. 

##### New Analysis

Young adults are still the most likely to survive but this time, middle-aged adults are the second group with the highest survival rate.

#### 4) Fare

In [24]:
fare_lowest = df_factors["Fare"].max()
print(f"Most expensive fare: ${fare_lowest}")
fare_highest = df_factors["Fare"].min()
print(f"Least expensive fare: ${fare_highest}")

Most expensive fare: $512.3292
Least expensive fare: $0.0


In [25]:
## Discretize variable into equal-sized buckets based on rank or sample quantiles
bin_labels_3 = ["Low", "Medium", "High"]
q3 = [0, 1/3, 2/3, 1]
df_factors["Fare Range"] = pd.qcut(df_factors["Fare"], q=q3, labels=bin_labels_3)
# bin_labels_4 = ["Quartile 1", "Quartile 2", "Quartile 3", "Quartile 4"]
# q4 = [0, 0.25, 0.5, 0.75, 1]
# df_factors["Fare Range"] = pd.qcut(df_factors["Fare"], q=q4, labels=bin_labels_4)
# bin_labels_6 = ["Low Low", "Low High", "Medium Low", "Medium High", "High Low", "High High"]
# q6 = [0, 1/6, 2/6, .5, 4/6, 5/6, 1]
# df_factors["Fare Range"] = pd.qcut(df_factors["Fare"], q=q6, labels=bin_labels_6)
print(df["Fare Range"].value_counts())

High      154
Medium     22
Low         7
Name: Fare Range, dtype: int64


##### Previous Analysis

Those passengers who bought the lowest-tier tickets were most likely to survive. But the difference between ticket tiers does not seem to be striking in terms of survival rate.

##### New Analysis

The new result is the opposite of the previous one. Now it shows that those passengers who bought the highest-tier tickets were most likely to survive. Perhaps they had the priority to board rescue boats.

#### 5) Embarkation

In [26]:
print(df_factors.groupby("Embarked")["Survived"].count())

Embarked
C     65
Q      2
S    116
Name: Survived, dtype: int64


##### Previous Analysis

Passengers who embarked at Southampton were significantly more likely to survive the disaster but those at Queenstown had the lowest chance of survival. 

##### New Analysis

The new result is similar to the previous one.